In [119]:
import numpy as np
import pandas as pd

In [120]:
reviews_df = pd.read_csv('basenotes_reviews_df.csv')

/Users/lucasreynolds/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [121]:
reviews_df.head()

,rating_value,review_id,user_id,user_name,user_location,scent_id,scent_name,review_text,scent_brand
0,2.0,232864,26195006,speedracer,United States,26160732.0,L'Homme Idéal Extrême,"Starts off very much like the original Ideal, ...",NaN
1,3.0,232854,26193350,epapsiou,Sikkim,26158303.0,Fougère L'Aube,Stardate 20200815:\n\r\nOpens up as bitter-gre...,NaN
2,3.0,232852,26210582,HouseOfPhlegethon,United States,26142534.0,Journey Woman,Jasmine almost smothers the apricot and osmant...,NaN
3,3.0,232851,26210582,HouseOfPhlegethon,United States,26153253.0,Eiderantler,Floral green. Wet greens. Sweet green notes....,NaN
4,3.0,232850,26210582,HouseOfPhlegethon,United States,26160913.0,Dinudisit (Sea Flowers),"Intense, sort of sweet flowers. Big narcissus...",NaN


In [122]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159324 entries, 0 to 159323
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   rating_value   159317 non-null  float64
 1   review_id      159324 non-null  object 
 2   user_id        159324 non-null  object 
 3   user_name      157520 non-null  object 
 4   user_location  127711 non-null  object 
 5   scent_id       159317 non-null  float64
 6   scent_name     159317 non-null  object 
 7   review_text    159293 non-null  object 
 8   scent_brand    159280 non-null  object 
dtypes: float64(2), object(7)
memory usage: 10.9+ MB


In [123]:
# pd.to_numeric(reviews_df['review_id'])

I attempted to convert review_id column to numeric data type, but there was an error. Looks like there is at least one review's text that's in that column. Some reviews must not have parsed into the correct fields.

## 2. Missing Values

In [124]:
# calculate null percentages
reviews_df.isnull().sum()/len(reviews_df)

rating_value     0.000044
review_id        0.000000
user_id          0.000000
user_name        0.011323
user_location    0.198420
scent_id         0.000044
scent_name       0.000044
review_text      0.000195
scent_brand      0.000276
dtype: float64

About 20% of user_location is null, but I noticed a lot of reviews didn't have user location listed so this isn't a big surprise.

About 1% of user_name is null, I wonder if Basenotes allows for anonymous reviews or if this is due to a data issue. I'll review this further in my next steps.

All other fields have less than 1% nulls. Rating_value, user_id, scent_id, and scent_name all have the same percentage of null values. I wonder if they are all due to the same rows.

### 2.1 Rating Value

In [125]:
# review rating values
reviews_df['rating_value'].value_counts(normalize=True)

3.0    0.637998
2.0    0.225337
1.0    0.135974
7.0    0.000690
Name: rating_value, dtype: float64

There should only be 3 rating values: 1 for thumbs down, 2 for thumb horizontal, and 3 for thumbs up. I'm not sure why there is a 7 for some values in this field. I want to sample some of these rows.

In [126]:
# sample rows with a rating value of 7
rating_value_7 = reviews_df[reviews_df['rating_value'] == 7]
rating_value_7.head()

,rating_value,review_id,user_id,user_name,user_location,scent_id,scent_name,review_text,scent_brand
145781,7.0,24732,5138,Strange Accord,United States,26123581.0,Bois et Musc,Lovely topnotes. Quickly enters nice buy weak...,Serge Lutens
146021,7.0,24702,5121,vinterdroppe,Sweden,10212395.0,Habanita,It´s hard for me to do either thumbs up or dow...,Molinard
146361,7.0,15711,3408,Naed_Nitram,NaN,26121250.0,O'Pomelo / Pamplemousse,The preliminary rendition of acerbic grapefrui...,Comptoir Sud Pacifique
146703,7.0,16196,3502,Nikolas@_,NaN,26120881.0,Joop! Homme,Homme by Joop!I actually had a girlfriend who...,Joop!
146760,7.0,13906,2971,mancheeros,NaN,26122372.0,Cuir de Russie,undoubtedly the 'dirtiest' of the Creed scents...,Creed


The rest of the data in these rows appears to have parsed correctly. I'm not exactly sure where they 7 came from. This is something I'll want to review and correct before any additional parsing. For now, I'm going to drop these rows.

In [127]:
# drop rows with rating value of 7
reviews_df = reviews_df[reviews_df['rating_value'] != 7]

# verify that rows with rating value of 7 were dropped
len(reviews_df[reviews_df['rating_value'] == 7])

0

In [130]:
# calculate number of rows with null rating value
reviews_df['rating_value'].isnull().sum()

7

In [131]:
# sample rows with null rating value
nulls_rating_value = reviews_df[reviews_df['rating_value'].isna()]
nulls_rating_value.head()

,rating_value,review_id,user_id,user_name,user_location,scent_id,scent_name,review_text,scent_brand
5943,NaN,This has truly turned out to be an intense eni...,Sultan Pasha,NaN,NaN,NaN,NaN,NaN,NaN
6511,NaN,The smell brings to mind rowing around little ...,Sultan Pasha,NaN,NaN,NaN,NaN,NaN,NaN
11730,NaN,Per Fumum: A Sanctified Rose is probably my pe...,Annette Neuffer,NaN,NaN,NaN,NaN,NaN,NaN
17552,NaN,Galantuomo is all about civilized spice – it’s...,Bottega Profumiera,NaN,NaN,NaN,NaN,NaN,NaN
19417,NaN,Sampled from a splash vial last night. Imagine...,Bottega Profumiera,NaN,NaN,NaN,NaN,NaN,NaN


The data for these reviews must not have parsed correctly. I'll go back and review/test my code to determine why this occurred, but for now I'm choosing to remove these since there are only 7 rows.

In [132]:
# drop rows with rating missing
reviews_df = reviews_df[reviews_df['rating_value'].notna()]

# verify null ratings were dropped
reviews_df['rating_value'].isna().sum()

0

### 2.2 User ID

In [133]:
# calculate the number of rows with null user id
reviews_df['user_id'].isna().sum()

0

The rows with null user_id must have also been the rows with null rating_value and were already removed.

### 2.3 User Name

In [134]:
# calculate number of rows with null user name
reviews_df['user_name'].isna().sum()

1797

In [135]:
# sample rows with null user name
nulls_user_name = reviews_df[reviews_df['user_name'].isna()]
nulls_user_name.head()

,rating_value,review_id,user_id,user_name,user_location,scent_id,scent_name,review_text,scent_brand
4040,3.0,223358,26278519,NaN,NaN,26141824.0,Oud Noir Intense,So I'm on my second day and second application...,Fragrance Du Bois
4322,3.0,222859,26278519,NaN,NaN,26138467.0,Dries Van Noten par Frederic Malle,"Dries Van Noten by FM , Wow, What a wonderfull...",Editions de Parfums Frederic Malle
7542,2.0,216417,26261143,NaN,NaN,26137673.0,L'Heure de Nuit,"Guerlain, why have you forsaken me? L’Heure B...",Guerlain
7543,3.0,216413,26261143,NaN,NaN,26131425.0,Les Heures de Parfum - XII L'Heure Mystérieuse,"Mysterious and bewitching, Cartier L’Heure Mys...",Cartier
9100,3.0,213539,26261143,NaN,NaN,26131761.0,Shem-el-Nessim,"*copied from my own review at Twisted Lily, da...",Grossmith


I'm curious whether the user IDs for these null usernames may be anywhere else in the dataframe and have the username. They could be used to fill these rows.

In [136]:
# create an array of the user IDs that are in the rows with null usernames
null_user_name_ids = nulls_user_name['user_id'].unique()

In [137]:
# see if any of the user IDs for the null usernames have usernames elsewhere
is_null_list = []

for id in null_user_name_ids:
    is_null = reviews_df[reviews_df['user_id'] == id]['user_name'].notna().sum()
    is_null_list.append(is_null)
    
sum(is_null_list)

0

There aren't any other rows with these user IDs that have usernames I can use to fill. For now I'll drop these rows. I may come back and look more into why these are null.

In [138]:
# drop rows with null user name
reviews_df = reviews_df[reviews_df['user_name'].notna()]

# verify the rows were dropped
reviews_df['user_name'].isna().sum()

0

### 2.4 User Location

In [139]:
# calculate the number of null user locations
reviews_df['user_location'].isna().sum()

29739

In [140]:
# sample rows with null user locations
nulls_user_location = reviews_df[reviews_df['user_location'].isna()]
nulls_user_location.head()

,rating_value,review_id,user_id,user_name,user_location,scent_id,scent_name,review_text,scent_brand
12,3.0,232776,26226431,Constantine,NaN,26155990.0,Century,Grapefruit cardamom musk sandalwood are the no...,NaN
19,1.0,232755,26226431,Constantine,NaN,26159222.0,Spice Blend,This smells like cheap nail varnish. Something...,NaN
22,3.0,232746,26270042,smooth_operator,NaN,26159550.0,Harrods Aoud,"As exceptional as Roja usually is, this is a r...",NaN
57,2.0,232585,26226431,Constantine,NaN,26147281.0,Sunshine Man,This is Amouage's take on Roja Dove's Risque. ...,Amouage
69,2.0,232544,26256039,framethephrases,NaN,26140106.0,No. 42 The Flower Shop,I'm giving this fragrance three stars. It's a ...,Jo Loves...


I noticed before parsing the reviews that not all users have a location listed. I'm curious if there is a relationship between user location and rating or between whether a user listed their location and rating so I don't want to drop the field. Instead I'll fill null user locations with no_location.

In [141]:
# fill null user locations with no_location
reviews_df['user_location'] = reviews_df['user_location'].fillna('no_location')

# verify that nulls were filled
reviews_df['user_location'].isna().sum()

0

### 2.5 Scent ID

In [142]:
num_null_scent_id = reviews_df['scent_id'].isna().sum()
print(f"Number of null scent IDs: {num_null_scent_id}")

Number of null scent IDs: 0


The rows with null scent IDs must have also been the rows with null ratings and were already dropped.

### 2.6 Scent Name

In [143]:
num_null_scent_name = reviews_df['scent_name'].isna().sum()
print(f"Number of null scent names: {num_null_scent_name}")

Number of null scent names: 0


The rows with null scent names must have also been the rows with null ratings. They were already dropped.

### 2.7 Review Text

In [144]:
# calcualate the number of rows with null review text
reviews_df['review_text'].isna().sum()

24

In [145]:
# sample some of the rows with null review text
null_review_text = reviews_df[reviews_df['review_text'].isna()]
null_review_text.head()

,rating_value,review_id,user_id,user_name,user_location,scent_id,scent_name,review_text,scent_brand
2759,3.0,225851,13447096,CleverUsername,no_location,26128887.0,17/17 Irisss,NaN,Xerjoff
4382,2.0,223510,3676,Paradeiserl,no_location,26159853.0,Halfeti Leather,NaN,Penhaligon's
5942,3.0,219573,13378905,Sniffers,United States,26148710.0,Enigma intense,NaN,NaN
6510,2.0,218441,13378905,Sniffers,United States,26148712.0,Via Dolorosa,NaN,NaN
11729,3.0,209155,4825,the_good_life,Germany,26150430.0,Per Fumum: A Sanctified Rose,NaN,NaN


The rows with null review appear to have all the other data. Rather than drop these I'm going to fill them with no_review_text.

In [146]:
# fill null review texts with no_review_text
reviews_df['review_text'] = reviews_df['review_text'].fillna('no_review_text')

# verify that all nulls were filled
reviews_df['review_text'].isna().sum()

0

### 2.8 Scent Brand

In [147]:
num_null_scent_brand = reviews_df['scent_brand'].isna().sum()
print(f"Number of rows with null scent_brand: {num_null_scent_brand}")

Number of rows with null scent_brand: 37


In [148]:
# sample some rows with null scent brand
nulls_scent_brand = reviews_df[reviews_df['scent_brand'].isna()]
nulls_scent_brand.head()

,rating_value,review_id,user_id,user_name,user_location,scent_id,scent_name,review_text,scent_brand
0,2.0,232864,26195006,speedracer,United States,26160732.0,L'Homme Idéal Extrême,"Starts off very much like the original Ideal, ...",NaN
1,3.0,232854,26193350,epapsiou,Sikkim,26158303.0,Fougère L'Aube,Stardate 20200815:\n\r\nOpens up as bitter-gre...,NaN
2,3.0,232852,26210582,HouseOfPhlegethon,United States,26142534.0,Journey Woman,Jasmine almost smothers the apricot and osmant...,NaN
3,3.0,232851,26210582,HouseOfPhlegethon,United States,26153253.0,Eiderantler,Floral green. Wet greens. Sweet green notes....,NaN
4,3.0,232850,26210582,HouseOfPhlegethon,United States,26160913.0,Dinudisit (Sea Flowers),"Intense, sort of sweet flowers. Big narcissus...",NaN


Since there are only 37 of these I am going to drop them for now.

In [149]:
# drop rows with null scent brand
reviews_df = reviews_df[reviews_df['scent_brand'].notna()]

# verify that nulls were dropped
reviews_df['scent_brand'].isna().sum()

0

In [150]:
# verify that no nulls remain
reviews_df.isna().sum()

rating_value     0
review_id        0
user_id          0
user_name        0
user_location    0
scent_id         0
scent_name       0
review_text      0
scent_brand      0
dtype: int64

In [151]:
# save cleaned dataframe to CSV
reviews_df.to_csv('basenotes_reviews_df_clean.csv', index=False)